In [ ]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [ ]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
]

In [ ]:
repo_path = "/Users/chanb/research/ualberta/simple_icl"
results_dir = "/Users/chanb/research/ualberta/simple_icl/results/simple_icl/results"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

In [ ]:
# variant_name = "simple_icl-fixed_g"
variant_name = "simple_icl-learned_g"

os.makedirs(
    os.path.join(repo_path, "cc_utils/agg_stats"),
    exist_ok=True
)
results_dir = os.path.join(results_dir, variant_name)

In [ ]:
stats_file = os.path.join(repo_path, "cc_utils/agg_stats", "{}.feather".format(variant_name))
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if repo_path.startswith("/Users"):
        it = tqdm(os.listdir(results_dir))
    else:
        it = os.listdir(results_dir)

    for run_name in it:
        tic = timeit.default_timer()

        prefix = "-".join(run_name.split("-")[:-9])
        seed = run_name.split("-")[-9]

        data = pickle.load(open(os.path.join(results_dir, run_name, "evaluation.pkl"), "rb"))
        for eval_name in data["stats"]:
            for stats_key in stats_keys:
                stats.append(
                    dict(
                        variant=prefix,
                        seed=seed,
                        eval_name=eval_name,
                        stats_key=stats_key,
                        stats=[float(val) for val in data["stats"][eval_name][stats_key]],
                        **{
                            "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                        },
                    )
                )

        toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(os.path.join(repo_path, "cc_utils/agg_stats", "{}.feather".format(variant_name)))

In [ ]:
len(stats)

In [ ]:
stats.info()

In [ ]:
stats